### Assignment 3

#### Flower classification

Adapt the method of Lecture 9 from binary classification to $C$-class classification ($C=5$) to classify images of flowers in [my repository](https://github.com/mgreenbe/flower-classification.git).

- Use an random 80%/20% split of the data into training and validation.

- Use a ResNet18, ResNet34, or a ResNet50 model, pretrained on ImageNet, with the classification head replaced as appropriate to our problem.

- At the end of each training epoch compute and print out the [cross-entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.functional.cross_entropy.html) and classification accuracy on the validation set.

- Time permitting, look into some tricks you might use to improve your classifier, e.g. [randomly flipping training images horizontally](https://pytorch.org/vision/0.20/generated/torchvision.transforms.RandomHorizontalFlip.html). (Why might this help?)

- You're also welcome to experiment with other vision models beyond the ResNets. If you find one that works particularly well, let me know!


In [8]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from torchvision.transforms import Compose, Normalize, Resize, ToTensor
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, resnet34
from pathlib import Path
from matplotlib import pyplot as plt
import random
from collections import Counter

In [2]:
transformations = Compose([
    Resize((128,128)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
ds = ImageFolder('assets', transform=transformations)

In [ ]:

# Get class names and their corresponding indices
class_to_idx = ds.class_to_idx  

class_counts = Counter(label for _, label in ds)

total_images = len(ds)

for class_name, class_idx in class_to_idx.items():
    count = class_counts[class_idx]
    share = (count / total_images) * 100  
    print(f"{class_name}: {count} images ({share:.2f}%)")

daisy: 769 images (17.79%)
dandelion: 1052 images (24.33%)
rose: 784 images (18.14%)
sunflower: 734 images (16.98%)
tulip: 984 images (22.76%)


Use an random 80%/20% split of the data into training and validation.

In [10]:

train_size = int(0.8 * len(ds))
val_size = len(ds) - train_size  # Ensures total matches dataset size

train_ds, val_ds = random_split(ds, [train_size, val_size])

In [11]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64)

In [12]:
def accuracy(yy, y):
  return torch.mean(1.0*(yy.argmax(dim=1) == y)).item()

In [15]:
# to resolve outdated certificate error
!pip install --upgrade certifi

    matplotlib-inline (<0.2.0appnope,>=0.1.0) ; platform_system == "Darwin"
                      ~~~~~~~~^


Use a ResNet18, ResNet34, or a ResNet50 model, pretrained on ImageNet, with the classification head replaced as appropriate to our problem.

In [33]:
model18 = resnet18(weights="IMAGENET1K_V1")
model18.fc = Linear(model18.fc.in_features, 5)

In [35]:
print(y.unique())

tensor([0, 1, 2, 3, 4])


In [36]:
opt = torch.optim.Adam(model18.parameters())

In [37]:
#model18.to("cuda")

loss_train = []
acc_train = []
for epoch in range(1):
  model18.train()
  for i, (X, y) in enumerate(train_dl):
    if i % 50 == 0:
      print(f"Processing batch {i} of {len(train_dl)}.")
    #X = X.to("cuda")
    #y = y.double().to("cuda")
    y = y.long()
    logits = model18(X)
    loss = F.cross_entropy(logits, y)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    loss_train.append(loss.detach().item())
    if i % 50 == 0:
      print(f"loss = {loss.detach().item()}.")

Processing batch 0 of 55.
loss = 1.803179144859314.
Processing batch 50 of 55.
loss = 0.5027329325675964.


ResNet34

In [38]:
model34 = resnet34(weights="IMAGENET1K_V1")
model34.fc = Linear(model34.fc.in_features, 5)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\Achu/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100.0%


In [39]:
opt = torch.optim.Adam(model34.parameters())

In [40]:
#model34.to("cuda")

loss_train = []
acc_train = []
for epoch in range(1):
  model34.train()
  for i, (X, y) in enumerate(train_dl):
    if i % 50 == 0:
      print(f"Processing batch {i} of {len(train_dl)}.")
    X = X
    y = y.long()
    logits = model34(X)
    loss = F.cross_entropy(logits, y)

    opt.zero_grad()
    loss.backward()
    opt.step()
    
    loss_train.append(loss.detach().item())
    if i % 50 == 0:
      print(f"loss = {loss.detach().item()}.")

Processing batch 0 of 55.
loss = 2.2447547912597656.
Processing batch 50 of 55.
loss = 0.7109651565551758.


At the end of each training epoch compute and print out the cross-entropy loss and classification accuracy on the validation set.

In [45]:
def evaluate(model, val_dl):
    loss_val = []
    acc_val = []
    model.eval()
    with torch.no_grad():
        for i, (X,y) in enumerate(val_dl):
            X= X
            y = y.long()
            logits = model(X)
            loss = F.cross_entropy(logits, y)
            loss_val.append(loss.item())
            _, predicted = torch.max(logits, dim=1)
            acc = torch.mean((predicted == y).double())
            acc_val.append(acc)
    return loss_val, acc_val


In [50]:
loss_val, acc_val = evaluate(model18,val_dl)

In [51]:
torch.mean(torch.Tensor(acc_val))

tensor(0.7461)

In [52]:
loss_val, acc_val = evaluate(model34,val_dl)

In [53]:
torch.mean(torch.Tensor(acc_val))

tensor(0.7138)

Randomly flipping trainingimages horizontally can improve the performance of the classifier because

1) Creaes new variation of the existing training image, thereby increasing diversity 
2) Avoid overfitting a specific orientation of the images 
3) The decision boundaries learned by the classifier becomes smoother, meaning model can classify instances with more flexibility and robustness